In [1]:
!pip install PyPDF2 pdf2image pytesseract pillow pydub tts
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
import re
!pip install gTTs
import google.generativeai as genai
import transformers
from pydub import AudioSegment
import os
from gtts import gTTS  # Using Google Text-to-Speech instead of OpenAI
!pip install google-generativeai python-pptx

import google.generativeai as genai
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor

!pip install moviepy
!pip install pydub

from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 81.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.5 MB

In [ ]:
genai.configure(api_key="API-KEY")

# **Text Extraction, Summary Generation and Simplification to Generate Scripts and Podcast**

gemini_mahida wadi file

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except:
        images = convert_from_path(pdf_path)
        for img in images:
            text += pytesseract.image_to_string(img)
    return text

def preprocess_text(text):
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

def summarize_with_gemini(text):
    model = genai.GenerativeModel('gemini-pro')
    prompt = """Summarize this research paper for a non-expert audience in a well-structured paragraph format.
    Do not use bullet points, numbering, asterisks, or bold text. Write naturally and cohesively while covering the key contributions, methodology, analysis, implications, limitations, and conclusion in a fluid and engaging manner.

    Paper: """ + text[:30000]

    response = model.generate_content(prompt)
    return response.text

def extract_key_concepts(text):
    # Using same BERT model as before
    nlp = transformers.pipeline("ner", model="dslim/bert-base-NER")
    entities = nlp(text)
    keywords = [entity["word"] for entity in entities if entity["entity"] in ["B-ORG", "B-MISC"]]
    return list(set(keywords))


In [5]:
def simplify_jargon(text):
    model = genai.GenerativeModel('gemini-pro')
    prompt = """Replace technical terms with simple analogies. Example:
    "Summary: This research paper is about optimizing resources in hospital departments through network slicing. The authors propose a method to assign individual network slices to each smart device in various hospital departments using a model named federated learning.

In the methodology, the network is divided into three types: Enhanced Mobile Broadband (eMBB), Ultra-Reliable Low Latency Communication (URLLC), and Massive Machine Type Communication (mMTC). All this to consider aspects such as bandwidth, data transmission speed, the number of devices supported at a time, and reliability. The method uses federated learning to ensure the privacy of patient data, keeping it within the respective department and not shared with others.

The results show that the model learns new patterns accurately (98% accuracy) with this real-time scenario-based approach leading to more efficient resource allocation. The results also show a consistent improvement in the accuracy rate and a decrease in the loss value with each round of learning, indicating the effectiveness of the model.

However, the study's limitation is that it doesn't include encryption techniques, which could further improve the safety of the model. Also, it's based on the assumption that data from all devices is periodically transmitted to a local model, which may not always be the case in actual scenarios.

In conclusion, the study proves the efficiency of federated learning and network slicing in smart healthcare facilities for optimizing resources and maintaining data privacy. The authors suggest that future research could enhance this model by integrating encryption techniques to improve patient data privacy even further." -> "Simplified text: This research paper is like a recipe for a better-run hospital using individual lanes for each smart gadget in different hospital departments, similar to having different checkout lines for different types of groceries. The recipe uses a model called federated learning, which is like a private tutor for each department, ensuring that patient data stays within that department and isn't shared with others.

The method splits the hospital's network into three types, like a highway with lanes for motorbikes, cars, and trucks, considering things like how wide the lanes are, how fast the vehicles can go, how many vehicles can fit in at a time, and how reliable the lanes are.

The outcomes show that this recipe works really well, with the model learning new routines accurately (98% accuracy) with this real-world test, leading to a smoother run hospital department. The outcomes also show a continuous increase in success rate and a decrease in errors with each learning session, showing the recipe works well.

However, the recipe's limitation is that it doesn't include a security guard (encryption techniques), which could make the model even safer. Also, it assumes that data from all gadgets is regularly sent to a local model, kind of like a department head, which may not always happen in real life.

To sum up, the study shows that the recipe using federated learning and network slicing works well in smart hospitals for making things run more smoothly and keeping patient data private. The authors suggest that future research could add a security guard to the recipe to keep patient data even safer."
    Text to simplify: """ + text

    response = model.generate_content(prompt)
    return response.text

def generate_analogy(concept):
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(f"Create a relatable analogy for this concept: {concept}")
    return response.text

In [6]:
def generate_video_script(summary: str, analogy: str, video_type) -> str:
    model = genai.GenerativeModel('gemini-pro')

    if video_type == "reel":
        structure = f"""
        - Hook (15 seconds): Grab attention with a surprising fact/question
        - Problem (30 seconds): Explain the research gap
        - Analogy (35 seconds): Simplify using {analogy}
        - Impact (30 seconds): Why this matters
        - Call-to-action (15 seconds)
        """
    else:
        structure = f"""
        - Intro (30s): Context + thesis
        - Methodology (90s): Non-technical explanation
        - Key Findings (90s): Visualized results
        - Real-World Example (60s): {analogy}
        - Conclusion (30s)
        """

    response = model.generate_content(
        f"Create a {video_type} script using this structure: {structure}\n"
        f"Content: {summary}"
    )
    return response.text

In [7]:
def generate_podcast_script(summary: str, analogy: str) -> str:
    """Generate podcast script with explicit analogy explanations"""
    model = genai.GenerativeModel('gemini-pro')

    prompt = f"""Create a podcast dialogue with these elements:
    1. Host asks questions about the research
    2. Expert explains concepts USING THIS ANALOGY: {analogy}
    3. For each technical concept:
       - First state the technical term
       - Then explain using the analogy
       - Finally connect back to the research

    Format exactly like:
    Host: [question]
    Expert: "[Technical concept] works similar to {analogy.split('→')[0].strip()}...
    (Explain analogy connection)
    This relates to our research because..."

    Include 5 question-answer pairs. Research summary: {summary[:15000]}"""

    try:
        response = model.generate_content(prompt)
        if not response.text:
            raise ValueError("Empty response from Gemini")
        return response.text
    except Exception as e:
        return fallback_script(summary, analogy)

def fallback_script(summary: str, analogy: str) -> str:
    """Guaranteed script with analogy explanation"""
    base_analogy = analogy.split('→')[0].strip()
    explanation = analogy.split('→')[1].strip()

    return f"""
    Host: Can you explain the core concept of this research?
    Expert: Certainly! {base_analogy} works similar to {explanation}.
    In this research, we've applied this principle to...

    Host: What makes this approach effective?
    Expert: Just like how {base_analogy} requires {explanation.split(' ')[0]},
    our method needs...

    Host: How does this compare to existing solutions?
    Expert: Traditional approaches work like [old method], but our {base_analogy}-inspired
    method {explanation}...

    Host: What real-world applications do you envision?
    Expert: Imagine using {base_analogy} for [application]. That's exactly how
    {explanation} could transform...

    Host: What's next for this research?
    Expert: We're expanding the {base_analogy} analogy to explore [new area],
    building on {explanation} fundamentals.
    """

def generate_podcast_audio(script: str, output_path: str = "podcast.mp3") -> str:
    """Generate audio with explicit analogy handling"""
    # Pre-process script
    script = script.strip()
    if not script:
        script = fallback_script("", "Technical process → Simple analogy")

    # Split roles with validation
    host_lines = []
    expert_lines = []

    for line in script.split('\n'):
        line = line.strip()
        if line.startswith("Host:"):
            host_lines.append(line.replace("Host:", "").strip())
        elif line.startswith("Expert:"):
            expert_content = line.replace("Expert:", "").strip()
            # Ensure analogy mention
            if "analogy" not in expert_content.lower():
                expert_content += f" (using our core analogy: {analogy})"
            expert_lines.append(expert_content)

    # Fallback if no lines found
    if not host_lines:
        host_lines = ["Could you explain the main concept using your analogy?"]
    if not expert_lines:
        expert_lines = [f"The key analogy is: {analogy}. This means..."]
    print(host_lines)
    print(expert_lines)
    # Generate voices
    try:
        # Host voice
        host_text = " ".join(host_lines)
        host_tts = gTTS(host_text, lang='en', slow=False)
        host_tts.save("host_temp.mp3")

        # Expert voice (slower pace)
        expert_text = " ".join(expert_lines)
        expert_tts = gTTS(expert_text, lang='en', slow=True)
        expert_tts.save("expert_temp.mp3")

        # Combine audio
        host_audio = AudioSegment.from_mp3("host_temp.mp3")
        expert_audio = AudioSegment.from_mp3("expert_temp.mp3")
        combined = host_audio + AudioSegment.silent(500) + expert_audio
        combined.export(output_path, format="mp3")

    finally:
        # Cleanup
        for f in ["host_temp.mp3", "expert_temp.mp3"]:
            if os.path.exists(f):
                os.remove(f)

    return output_path

In [9]:
paper_text = extract_text_from_pdf("/content/Lakshit___Quantum__Riya__Param.pdf")
clean_text = preprocess_text(paper_text)
summary = summarize_with_gemini(clean_text)
print(summary)
keywords = extract_key_concepts(clean_text)
simplified_text = simplify_jargon(summary)
print(simplified_text)
analogy = generate_analogy("Generative Adversarial Networks")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This paper presents a novel framework for Healthcare IoT (H-IoT) networks that combines quantum communication, edge intelligence, and machine learning for secure and efficient data transmission.

**Methodology:**
The framework utilizes a four-layer architecture:
1. **Wearable Health Edge Layer:** Collects real-time patient data from wearable sensors.
2. **Network Intrusion Detection Layer:** Detects network intrusions using machine learning models, with Random Forest (RF) found to be most effective.
3. **Quantum Layer:** Employs quantum cryptographic protocols (BB84 and E91) to encrypt healthcare data, ensuring data security and integrity.
4. **Hospital Layer:** Receives and processes secure healthcare data, facilitating real-time monitoring and enhancing patient safety.

**Analysis:**
The framework was evaluated on a dataset of network traffic related to Wireless Body Area Networks (WBANs). RF outperformed other machine learning models in detecting intrusions, achieving an accuracy of

Device set to use cuda:0


**Simplified Text:**

Imagine a smart hospital with wearable devices that track patients' health. This information is like a valuable treasure that needs to be kept safe.

To protect this treasure, we have a team of experts (edge intelligence) who use special computers (machine learning) to search for "bad guys" (network intrusions). Like a super spy, they use a secret code (quantum cryptography) to keep the treasure safe.

The treasure is then sent to the hospital's headquarters (hospital layer), where it's used to make sure patients are safe.

This system has been tested, and it's like a superhero with a 90% success rate at catching the "bad guys." And it uses the secret code to keep the treasure safe.

This system helps hospitals run more smoothly, protect patient information, and improve healthcare. But it's still like a new invention that needs some improvements, like making sure it can be used in every hospital and finding ways to make it work even faster.

In the future, we'll m

In [10]:
script = generate_podcast_script(summary, analogy)
script = script.replace("*", "").replace("", "")
print("Generated Script:\n", script)
audio_file = generate_podcast_audio(script)
print("Podcast saved to:", audio_file)

Generated Script:
 Host: Could you elaborate on the role of Generative Adversarial Networks (GANs) in H-IoT networks?

Expert: GANs follow a concept similar to Generative Adversarial Networks (GANs): The Art Studio Analogy.

Imagine an artist's studio with two students:

 Generator (Alice): A talented painter who can create beautiful artwork but lacks the skill to distinguish between a masterpiece and a forgery.
 Discriminator (Bob): A highly experienced art critic who can quickly spot the difference between a genuine masterpiece and a fake.

GAN Process:

 Training: Alice (Generator) paints countless variations of artwork. Bob (Discriminator) evaluates each one, giving feedback to Alice about what distinguishes real from fake.
 Iteration: Alice uses Bob's feedback to improve her painting skills, gradually creating artwork that becomes indistinguishable from real paintings.
 Outcome: In the end, Alice becomes so skilled that her paintings fool even Bob, the expert.

How it Relates to G

# **PPT Generation**

In [17]:
# Define theme styles
THEMES = {
    "formal": {
        "title_font": "Times New Roman",
        "title_size": Pt(44),
        "title_color": RGBColor(0,0,0),  # Black
        "title_bold": True,
        "title_all_caps": True,
        "body_font": "Times New Roman",
        "body_size": Pt(20),
        "body_color": RGBColor(92, 64, 51),  # Black
        "background_color": RGBColor(196, 164, 132),  # Gray
    },
    "casual": {
        "title_font": "Comic Sans MS",
        "title_size": Pt(40),
        "title_color": RGBColor(255, 165, 0),  # Blue
        "body_font": "Segoe Script",
        "body_size": Pt(18),
        "body_color": RGBColor(0,0,0),  # Black
        "background_color": RGBColor(237,232,208),  # Light yellow
    },
    "fun": {
        "title_font": "Impact",
        "title_size": Pt(48),
        "title_color": RGBColor(136, 8, 8),  # Pink
        "body_font": "Verdana",
        "body_size": Pt(22),
        "body_color": RGBColor(0,0,0),  # Black
        "background_color": RGBColor(186,225,255),  # Light green
    }
}

In [18]:
def generate_slides_content(summary):
    """
    Generates structured PowerPoint slide content using Gemini API.
    Returns formatted text with slide titles and bullet points.
    """
    prompt = f"""Create an 8-slide PowerPoint presentation from this research paper summary.
    Ensure every slide follows this structure:

    Slide 1: Title Slide
    - **Title**: [Research Paper Title]
    - **Presented by:**: "Presented by: Deep Drillers"

    Slide 2: Introduction
    - Background of the study
    - Research objectives
    - Key questions addressed

    Slide 3: Methodology
    - Research design
    - Data collection methods
    - Analysis techniques

    Slide 4: Key Findings
    - Main result 1
    - Main result 2
    - Main result 3

    Slide 5: Analysis
    - Interpretation of results
    - Comparison with previous work
    - Statistical significance

    Slide 6: Implications
    - Theoretical implications
    - Practical applications
    - Policy recommendations

    Slide 7: Limitations
    - Study limitations 1
    - Study limitations 2
    - Future research directions

    Slide 8: Conclusion
    - Summary of key points
    - Final remarks
    - Acknowledgments

    Paper Summary: {summary}

    **Ensure slides are formatted with 'Slide X: Title' followed by bullet points ('- ').**
    """

    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)

    slide_content = response.text.strip()  # Ensure clean formatting

    # Remove the asterisks from the slide content
    slide_content = slide_content.replace("**", "")

    print("Generated Slide Content:\n", slide_content)
    return slide_content

In [19]:
def parse_ai_content(ai_output):
    """Convert raw AI text output to structured format"""
    slides = []
    current_slide = {"title": "", "bullets": []}

    for line in ai_output.split("\n"):
        line = line.strip()
        if line.startswith("Slide"):
            if current_slide["title"]:
                slides.append(current_slide)
                current_slide = {"title": "", "bullets": []}
            current_slide["title"] = line.split(": ")[1]
        elif line.startswith("-"):
            current_slide["bullets"].append(line[2:])

    if current_slide["title"]:
        slides.append(current_slide)

    return slides

In [20]:
def create_presentation(content, theme_name, filename="research_presentation.pptx"):
    """Main function to create a PowerPoint presentation with a selected theme"""
    # Parse raw AI content
    structured_content = parse_ai_content(content)

    # Create a blank presentation
    prs = Presentation()

    # Get the selected theme
    theme = THEMES.get(theme_name, THEMES["formal"])  # Default to "formal" if theme not found

    # Create slides
    for slide_data in structured_content:
        slide = prs.slides.add_slide(prs.slide_layouts[1])  # Use a blank slide layout
        title = slide.shapes.title
        title.text = slide_data["title"].upper() if theme.get("title_all_caps", False) else slide_data["title"]
        title.text_frame.paragraphs[0].font.name = theme["title_font"]
        title.text_frame.paragraphs[0].font.size = theme["title_size"]
        title.text_frame.paragraphs[0].font.color.rgb = theme["title_color"]
        title.text_frame.paragraphs[0].font.bold = theme.get("title_bold", False)

        content_box = slide.shapes.placeholders[1]
        for bullet in slide_data["bullets"]:
            p = content_box.text_frame.add_paragraph()
            p.text = bullet
            p.level = 0
            p.font.name = theme["body_font"]
            p.font.size = theme["body_size"]
            p.font.color.rgb = theme["body_color"]

        # Set slide background color
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = theme["background_color"]

    # Add footer
    for slide in prs.slides:
        txBox = slide.shapes.add_textbox(Inches(0.5), Inches(6.8), Inches(9), Inches(0.4))  # Adjusted position and size
        tf = txBox.text_frame
        tf.text = "Made by DesAI"
        tf.paragraphs[0].font.size = Pt(12)
        tf.paragraphs[0].font.color.rgb = RGBColor(150, 150, 150)  # Gray
        tf.paragraphs[0].alignment = 1  # Center alignment

    # Save the presentation
    prs.save(filename)
    return filename

In [24]:
if __name__ == "__main__":
    # Ask the user to select a theme
    print("Select a theme for your presentation:")
    print("1. Formal")
    print("2. Casual")
    print("3. Fun")
    theme_choice = input("Enter the number corresponding to your choice (1/2/3): ").strip()

    # Map user input to theme name
    theme_mapping = {"1": "formal", "2": "casual", "3": "fun"}
    chosen_theme = theme_mapping.get(theme_choice, "formal")  # Default to "formal" if invalid input

    paper_summary = summary
    # Generate slide content
    print("Generating slide content...")
    content = generate_slides_content(paper_summary)
    print("Content generated!\n")

    # Create PowerPoint
    print("Creating presentation...")
    output_file = create_presentation(content, chosen_theme)
    print(f"Presentation saved as {output_file}")

    # If using Google Colab, download the file
    try:
        from google.colab import files
        files.download(output_file)
    except ImportError:
        print("Run this in Colab to download the file automatically.")

Select a theme for your presentation:
1. Formal
2. Casual
3. Fun
Enter the number corresponding to your choice (1/2/3): 3
Generating slide content...
Generated Slide Content:
 Slide 1: Title Slide

- Title: Retrieval Augmented Generative AI Chatbot for Healthcare
- Presented by: Deep Drillers

Slide 2: Introduction

- Background: Generative AI in healthcare has limitations ("productivity paradox")
- Research objectives: Develop a RAG AI Chatbot framework to overcome these limitations
- Key questions addressed: Can RAG enhance consultation summaries, diagnostic insights, and emotion assessments?

Slide 3: Methodology

- Research design: Data-driven framework integrating multiple modules
- Data collection methods: Pre-existing medical knowledge databases
- Analysis techniques: Machine learning models, natural language processing

Slide 4: Key Findings

- Main result 1: Comprehensive patient summaries with clinical entities highlighted
- Main result 2: ML-based disease diagnosis predictio

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Video Generation**

In [16]:
def process_videos(video_files, audio_file, output_dir="output"):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load the audio file
    audio_clip = AudioFileClip(audio_file)
    audio_duration = audio_clip.duration

    for i, video_file in enumerate(video_files):
        # Load the video file
        video_clip = VideoFileClip(video_file)

        # Cut the video to the duration of the audio
        video_clip = video_clip.subclip(0, min(video_clip.duration, audio_duration))

        # If the video has its own audio, mix it with the provided audio
        if video_clip.audio is not None:
            final_audio = CompositeAudioClip([video_clip.audio, audio_clip])
        else:
            final_audio = audio_clip

        # Set the final audio to the video
        final_clip = video_clip.set_audio(final_audio)

        # Save the final video
        output_file = os.path.join(output_dir, f"output_video_{i+1}.mp4")
        final_clip.write_videofile(output_file, codec="libx264")

        # Close the clips to free up resources
        video_clip.close()
        final_clip.close()

    audio_clip.close()

if __name__ == "__main__":
    # List of video files (you can modify this to upload files via a GUI or other means)
    #video_files = ["video1.mp4", "video2.mp4", "video3.mp4", "video4.mp4", "video5.mp4"]

    # Audio file to overlap with the videos
    audio_file = "/content/podcast.mp3"

    # Process the videos
    process_videos(["/content/video.mp4"], audio_file)

Moviepy - Building video output/output_video_1.mp4.
MoviePy - Writing audio in output_video_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/output_video_1.mp4



Moviepy - Done !
Moviepy - video ready output/output_video_1.mp4
